# 1. Enviar os dados para o hdfs

    a. Fazer o download do arquivo no diretório input do SPARK (comandos rodados no terminal Linux)
    
sudo mkdir spark/input/covid
cd covid
sudo curl -O https://s3-sa-east-1.amazonaws.com/ckan.saude.gov.br/PNI/vacina/uf/2021-11-05/uf%3DSP/part-00000-aa6d7a64-f91d-43b4-afe9-09bc6c16a91e.c000.csv


     b. O arquivo baixado foi enviado para o hdfs (comandos rodados no terminal Linux)
     
hdfs dfs -put /input/covid/part-00000-aa6d7a64-f91d-43b4-afe9-09bc6c16a91e.c000.csv /user/carlos/projeto/

    c. O arquivo foi renomeado para facilitar a codificação do projeto (comandos rodados no terminal Linux)
   
hdfs dfs -mv /user/carlos/data/part-00000-aa6d7a64-f91d-43b4-afe9-09bc6c16a91e.c000.csv /user/carlos/projeto/covid.csv


In [1]:
# Confirmação da cópia e renomeação do arquivo.

!hdfs dfs -ls /user/carlos/projeto/covid.csv

-rw-r--r--   3 root supergroup   32587776 2021-11-08 09:30 /user/carlos/projeto/covid.csv


In [ ]:
# Visualização do arquivo.

!hdfs dfs -tail /user/carlos/projeto/covid.csv

# 2. Otimizar todos os dados do hdfs para uma tabela Hive particionada por município.

    a. Acessar o serviço hive-server (comandos rodados no terminal Linux)
docker exec -it hive-server bash

    b. Conectar o hive-serve utilizando o Beeline (comandos rodados no terminal Linux):
beeline -u 'jdbc:hive2://localhost:10000 -uroot'
SET hive.exec.dynamic.partition = true;
SET hive.exec.dynamic.partition.mode = nonstrict ;

    c. Verificar os databases e criar o database "vacinacao" e a tabela "vacinasp" (comandos rodados no terminal Linux):
show databases;
create database vacinacao;
use vacinacao;
create external table vacinasp
    (
    document_id varchar(50) comment 'ID do Documento do Paciente', 
    paciente_id varchar(60) comment 'ID do Paciente', 
    paciente_idade smallint comment 'Idade do Paciente', 
    paciente_data_nasc string, 
    paciente_enumSexoBiologico varchar(1) comment 'Sexo do Paciente', 
    paciente_raca_cod smallint, 
    paciente_raca_valor string, 
    paciente_cod_municipio int comment 'Codigo do Municipio', 
    paciente_pais int, 
    paciente_municipio string comment 'Nome do Municipio', 
    paciente_pais_nome string, 
    paciente_uf string, 
    paciente_cep string, 
    paciente_nacionalidade string, 
    estabelecimento_valor int, 
    estabelecimento_razao string, 
    estabelecimento_fantasia string, 
    estabelecimento_cod_mun int, 
    estabelecimento_municipio string comment 'Municipio de Aplicacao da Vacina', 
    estabelecimento_uf string, 
    vacina_grupo_cod int comment 'Codigo do Grupo Vacinal', 
    vacina_grupo_nome varchar(30) comment 'Nome do Grupo Vacinal', 
    vacina_categoria_cod int, 
    vacina_categoria_nome string, 
    vacina_lote string, 
    vacina_fabricante_nome varchar(30) comment 'Fabricante da Vacina', 
    vacina_fabricante_ref string, 
    vacina_dataAplicacao date comment 'Data de Aplicacao da Vacina', 
    vacina_descricao_dose string comment 'Numero da Dose', 
    vacina_cod int, 
    vacina_nome varchar(50) comment 'Nome da Vacina', 
    sistema_origem string, 
    time_stamp string, 
    desconhecido string 
    ) 
    row format delimited 
    fields terminated by ';' 
    lines terminated by '\n' 
    location '/user/carlos/projeto/'
    tblproperties('skip.header.line.count'='1')
    ;
    
    d. Criar tabela particionada por municipio (comandos rodados no terminal Linux):

create external table vacinasp_part
    (
    document_id varchar(50) comment 'ID do Documento do Paciente', 
    paciente_id varchar(60) comment 'ID do Paciente', 
    paciente_idade smallint comment 'Idade do Paciente', 
    paciente_enumSexoBiologico varchar(1) comment 'Sexo do Paciente',
    paciente_cod_municipio int comment 'Codigo do Municipio', 
    vacina_grupo_cod int comment 'Codigo do Grupo Vacinal',
    vacina_grupo_nome varchar(30) comment 'Nome do Grupo Vacinal', 
    vacina_fabricante_nome varchar(30) comment 'Fabricante da Vacina', 
    vacina_dataAplicacao date comment 'Data de Aplicacao da Vacina', 
    vacina_descricao_dose string comment 'Numero da Dose',
    vacina_nome varchar(50) comment 'Nome da Vacina'
    ) 
    partitioned by (estabelecimento_municipio string comment 'Municipio de Aplicacao da Vacina')
    location '/user/carlos/projeto/'
    ;
    
    e. Inserir dados (comandos rodados no terminal Linux)
    
insert overwrite table vacinasp_part partition (estabelecimento_municipio) 
    select document_id, 
            paciente_id, 
            paciente_idade, 
            paciente_enumSexoBiologico, 
            paciente_cod_municipio, 
            estabelecimento_municipio,
            vacina_grupo_cod, 
            vacina_grupo_nome, 
            vacina_fabricante_nome, 
            vacina_dataAplicacao, 
            vacina_descricao_dose, 
            vacina_nome 
    from vacinasp;    
    
    
!hdfs dfs -ls /user/carlos/projeto

In [2]:
!hdfs dfs -ls /user/carlos/projeto

Found 5 items
-rw-r--r--   3 root supergroup   32587776 2021-11-08 09:30 /user/carlos/projeto/covid.csv
drwxr-xr-x   - root supergroup          0 2021-11-08 10:51 /user/carlos/projeto/estabelecimento_municipio=%22COVID-19 ASTRAZENECA%2FFIOCRUZ - COVISHIELD%22
drwxr-xr-x   - root supergroup          0 2021-11-08 10:51 /user/carlos/projeto/estabelecimento_municipio=%22COVID-19 PFIZER - COMIRNATY%22
drwxr-xr-x   - root supergroup          0 2021-11-08 10:51 /user/carlos/projeto/estabelecimento_municipio=%22COVID-19 SINOVAC%2FBUTANTAN - CORONAVAC%22
drwxr-xr-x   - root supergroup          0 2021-11-08 10:51 /user/carlos/projeto/estabelecimento_municipio=__HIVE_DEFAULT_PARTITION__


# Rodrigo, realmente não sei dizer o que acontece. Tentei baixar o arquivo que estava no pdf do projeto, mas ele vem com problema e não abre.
# A partir daí, baixei o arquivo de vacinação do estado de SP, cerca de 32GB de informação, motivo pelo qual, "ACREDITO" que os comandos não tenham funcionado devido à baixa capacidade de processamento e memória.

# 3. Criar as 3 vizualizações pelo Spark com os dados enviados para o HDFS

In [ ]:
from pyspark.sql.types import *
columns_list = [
    StructField("document_id", StringType() ), 
    StructField("paciente_id", StringType() ),
    StructField("paciente_idade", StringType() ),
    StructField("paciente_data_nasc", StringType() ),
    StructField("paciente_enumSexoBiologico", StringType() ),
    StructField("paciente_raca_cod", StringType() ),
    StructField("paciente_raca_valor", StringType() ),
    StructField("paciente_cod_municipio", StringType() ),
    StructField("paciente_pais", StringType() ),
    StructField("paciente_municipio", StringType() ),
    StructField("paciente_pais_nome", StringType() ),
    StructField("paciente_uf", StringType() ),
    StructField("paciente_cep", StringType() ),
    StructField("paciente_nacionalidade", StringType() ),
    StructField("estabelecimento_valor", StringType() ),
    StructField("estabelecimento_razao", StringType() ),
    StructField("estabelecimento_fantasia", StringType() ),
    StructField("estabelecimento_cod_mun", StringType() ),
    StructField("estabelecimento_municipio", StringType() ),
    StructField("estabelecimento_uf", StringType() ),
    StructField("vacina_grupo_cod", StringType() ),
    StructField("vacina_grupo_nome", StringType() ),
    StructField("vacina_categoria_cod", StringType() ),
    StructField("vacina_categoria_nome", StringType() ),
    StructField("vacina_lote", StringType() ),
    StructField("vacina_fabricante_nome", StringType() ),
    StructField("vacina_fabricante_ref", StringType() ),
    StructField("vacina_dataAplicacao", StringType() ),
    StructField("vacina_descricao_dose", StringType() ),
    StructField("vacina_cod", StringType() ),
    StructField("vacina_nome", StringType() ),
    StructField("sistema_origem", StringType() ),
    StructField("time_st", StringType() ),
    StructField("desconhecido", StringType() )
                ]
covid_schema = StructType(columns_list)
covid_df = spark.read.option("header","true").schema(covid_schema).csv("/user/carlos/projeto/covid.csv")

In [ ]:
covid_df.show(5)

# Rodrigo, mais uma vez não consigo fazer o exercício e não sei o motivo das falhas. 
# Infelizmente encerro aqui minhas tentativas de executar o projeto.


4. Salvar a primeira visualização como tabela Hive
5. Salvar a segunda visualização com formato parquet e compressão snappy
6. Salvar a terceira visualização em um tópico no Kafka
7. Criar a visualização pelo Spark com os dados enviados para o HDFS
8. Salvar a visualização do exercício 6 em um tópico no Elastic
9. Criar um dashboard no Elastic para visualização dos novos dados enviados